# Classical Component

In [ ]:
import qsharp
# qsharp.reload() helped when couldn't locate any namespaces
# qsharp.get_available_operations_by_namespace() was good for debugging

In [ ]:
qsharp.get_available_operations_by_namespace()

In [ ]:
qsharp.packages.add("Microsoft.Quantum.Chemistry")
# packs = qsharp.Packages(qsharp.client)
# packs.add(package_name="Microsoft.Quantum.Chemistry")

In [ ]:
qsharp.reload()
qsharp.get_available_operations_by_namespace()

In [ ]:
from SLAP import VQEGetEnergy, PEAGetEnergy

In [ ]:
# from qdk.chemistry import Molecule
from qdk.chemistry.broombridge import load_and_encode

In [ ]:
# caffine = Molecule.from_xyz("data/caffeine/caffeine.xyz")

In [ ]:
# caffine.mol

In [ ]:
from scipy.optimize import minimize

def VQEGetEnergyWrapper(var_params, molecule_data, num_samples) -> float:
    theta1, theta2, theta3 = var_params
    return VQEGetEnergy.simulate(
        JWEncodedData=molecule_data,
        theta1=theta1,
        theta2=theta2,
        theta3=theta3,
        nSamples=num_samples
    )

def VQE_Classical_Routine(molecule_data, initial_var_params, num_samples):
    """ Run VQE Optimization to find the optimal energy and the associated variational parameters """
    opt_result = minimize(
        VQEGetEnergyWrapper,
        initial_var_params,
        args=(molecule_data, num_samples,),
        method="COBYLA",
        tol=0.000001,
        options={'disp': True, 'maxiter': 200,'rhobeg' : 0.05}
    )

    if opt_result.success:
        print(opt_result.message)
        print(f"Lowest Energy: {opt_result.fun} Ha")
        print(f"Number of evaluations: {opt_result.nfev}")
        print(f"Optimal parameters found: {opt_result.x}")

    return opt_result

In [ ]:
encoded_data = load_and_encode("data/H2/H2.yaml")

In [ ]:
# encoded_data

In [ ]:
VQE_Classical_Routine(encoded_data, [0.001, -0.001, 0.001], 1)

In [ ]:
VQEGetEnergy.simulate(
    JWEncodedData=encoded_data,
    theta1=0.001,
    theta2=-0.001,
    theta3=0.001,
    nSamples=100
)

In [ ]:
# VQEGetEnergy.estimate_resources(
#     JWEncodedData=encoded_data,
#     theta1=0.001,
#     theta2=-0.001,
#     theta3=0.001,
#     nSamples=1
# )

In [ ]:
PEAGetEnergy.simulate(
    qSharpData=encoded_data,
    nBitsPrecision=10,
    trotterStepSize=0.2,
    trotterOrder=1
)

In [ ]:
PEAGetEnergy.estimate_resources(
    qSharpData=encoded_data,
    nBitsPrecision=10,
    trotterStepSize=0.2,
    trotterOrder=1
)

In [ ]:
res = {}
for i in [1, 10, 50, 100, 250, 500, 1000, 5000, 10000, 1000000]:
    res[i] = VQEGetEnergy.simulate(
        JWEncodedData=encoded_data,
        theta1=0.001,
        theta2=-0.001,
        theta3=0.001,
        nSamples=i
    )
    print(i)

In [ ]:
res